In [3]:
from scipy import spatial
import pandas as pd


## Load Embedded Vectors

In [4]:
df=pd.read_csv('D:\Embedding\Code\Data\Processrd_Data\EmbeddedVecor_Coronary.csv')

## Calculate Cosine Distance for Embedded Vectors

In [5]:
list_Dis=[]
for i in range(len(df)):
    dataSetI=list(df.iloc[i][0:19])
    for j in range(len(df)):
        dataSetII=list(df.iloc[j][0:19])
        list_Dis.append([i,j,1 - spatial.distance.cosine(dataSetI, dataSetII)])

In [6]:
df_dis=pd.DataFrame(list_Dis,columns=['p1','p2','distance'])
df_dis

,p1,p2,distance
0,0,0,1.000000
1,0,1,0.082137
2,0,2,0.199644
3,0,3,-0.172991
4,0,4,0.149356
...,...,...,...
2111,45,41,0.783647
2112,45,42,0.681884
2113,45,43,0.663566
2114,45,44,0.783986


## Sort patients based on cosine distance DESC
### Retrive Top_K for query patient Q_i

In [86]:

list_patient=df_dis['p1'].unique().tolist()
ls_ranked=df_dis[df_dis['p1']==Q_i].sort_values('distance',ascending=False)
ls_ranked.reset_index()
ls_ranked_K=ls_ranked.iloc[0:k-1]


In [161]:
k=10
Q_i=100016
ls_ranked_K2=pd.read_csv("D:\\Embedding\\Code\\Data\\test.csv")

ls_ranked_K2=ls_ranked_K2[ls_ranked_K2['p1']==Q_i].sort_values('distance',ascending=False)
ls_ranked_K2.reset_index()
retrive_score=list(ls_ranked_K2['distance'][1:k].round(4))
retrive_score

[0.8232, 0.8052, 0.65, 0.5896, 0.5739, 0.5681, 0.5524, 0.5303]

# MSE


highest gold standard SimIndex score is identied and the score recorded as Yi
given a top-k ranking produced by a patient similarity method, the score Yi_Estimated is computed  as max(SimIndex(Qi; P1); ... ; SimIndex(Qi; Pk))

## Estimated_Y_i

In [40]:
ls_Estimated_Y=[]
for i in range(len(ls_ranked_K2)) :
    p1=ls_ranked_K2.iloc[i]['p1']
    p2=ls_ranked_K2.iloc[i]['p2']
    ls_Estimated_Y.append([p2,float(gold_dff[(gold_dff['p1']==p1)  & (gold_dff['p2']==p2)].SimIndex)])

    

In [47]:
# take second element for sort
def takeSecond(elem):
    return elem[1]

In [60]:

# sort list with key
ls_Estimated_Y.sort(key=takeSecond,reverse = True)

# print list
Estimated_Y_i= ls_Estimated_Y[0][1]
Estimated_Y_i

0.1236

## Y_i

In [78]:
#Load Gold Standard
gold_dff=pd.read_csv('D:\Embedding\Code\gold_standard_Full.csv')
gold_dff.head()


,p1,p2,SimIndex
0,100016,100016,1.0000
1,100016,100050,0.0000
2,100016,100058,0.0000
3,100016,100074,0.1236
4,100016,100109,0.0187


In [150]:
Q_i=100016
gold_Y_i=gold_dff[gold_dff['p1']==Q_i].sort_values('SimIndex',ascending=False)
gold_Y_i=gold_Y_i.iloc[0:k-1]
gold_Y_i.reset_index()
ideal=list(gold_Y_i['SimIndex'][1:k])
ideal

[0.8163, 0.8163, 0.8061, 0.8051, 0.7992, 0.7992, 0.7985, 0.7985]

In [80]:
Y_i=gold_Y_i.iloc[1].SimIndex
Y_i.to_numpy()

0.8163

## Calculate MSE

In [62]:
 import numpy as np

In [81]:

MSE = np.square(np.subtract(Y_i,Estimated_Y_i)).mean() 
MSE

0.47983328999999997

## Calculate Discounted Cumulative Gain  

In [151]:
ideal

[0.8163, 0.8163, 0.8061, 0.8051, 0.7992, 0.7992, 0.7985, 0.7985]

In [162]:
retrive_score

[0.8232, 0.8052, 0.65, 0.5896, 0.5739, 0.5681, 0.5524, 0.5303]

In [163]:
DCG_ideal=0
for i in range(1,len(ideal)+1):
    DCG_ideal=DCG_ideal+(ideal[i-1]/(np.log(i+1)/np.log(2)))
print(DCG_ideal)


3.1930345730039167


In [164]:
DCG_retrive=0
for i in range(1,len(retrive_score)+1):
    DCG_retrive=DCG_retrive+(retrive_score[i-1]/(np.log(i+1)/np.log(2)))
print(DCG_retrive)

2.6859520227765863


In [165]:
nDCG=DCG_retrive/DCG_ideal
nDCG

0.8411910242016949